In [11]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
from io import StringIO
import geopandas as gpd
import folium

In [12]:
#Incidents = pd.read_csv("../Metro_Nashville_Police_Department_Incidents.csv")

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [30]:
dates = {"$where":'incident_reported between "2021-01-01T00:00:00.000" and "2021-06-30T23:59:59:000"'}
response = requests.get('https://data.nashville.gov/resource/2u6v-ujjs.json', params = dates)

#ttps://data.cityofchicago.org/resource/6zsd-86xi.json?$where=date between '2015-01-10T12:00:00' and '2015-01-10T14:00:00'



In [31]:
response.text

'{"message":"Query coordinator error: query.soql.type-mismatch; Type mismatch for #BETWEEN, is text; position: Map(row -> 1, column -> 651, line -> \\"SELECT `primary_key`, `incident_number`, `report_type`, `report_type_description`, `incident_status_code`, `incident_status_description`, `investigation_status`, `incident_occurred`, `incident_reported`, `incident_location`, `latitude`, `longitude`, `zip_code`, `rpa`, `zone`, `location_code`, `location_description`, `offense_number`, `offense_nibrs`, `offense_description`, `weapon_primary`, `weapon_description`, `victim_number`, `domestic_related`, `victim_type`, `victim_description`, `victim_gender`, `victim_race`, `victim_ethnicity`, `victim_county_resident`, `mapped_location` WHERE `incident_reported` BETWEEN \'2021-01-01T00:00:00.000\' AND \'2021-06-30T23:59:59:000\'\\\\n                                                                                                                                                                     

2. Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.

In [7]:
#gpd.read_file("../tl_2019_47_tract/tl_2019_47_tract.shp")  
gpd.read_file(r"C:\Users\j0909\Desktop\tl_2019_47_tract\tl_2019_47_tract.shp")

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,165,021002,47165021002,210.02,Census Tract 210.02,G5020,S,30654374,207994,+36.3521602,-086.6464309,"POLYGON ((-86.69927 36.32775, -86.69923 36.327..."
1,47,165,020901,47165020901,209.01,Census Tract 209.01,G5020,S,23811766,3412697,+36.3483143,-086.4070907,"POLYGON ((-86.43949 36.35962, -86.43915 36.359..."
2,47,165,020300,47165020300,203,Census Tract 203,G5020,S,14629292,0,+36.5796910,-086.5114955,"POLYGON ((-86.53716 36.56852, -86.53713 36.568..."
3,47,165,020902,47165020902,209.02,Census Tract 209.02,G5020,S,9756715,0,+36.3724350,-086.4411274,"POLYGON ((-86.47106 36.38238, -86.47081 36.382..."
4,47,165,020800,47165020800,208,Census Tract 208,G5020,S,10235406,37743,+36.4005117,-086.4528100,"POLYGON ((-86.48786 36.37320, -86.48783 36.373..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,47,157,022112,47157022112,221.12,Census Tract 221.12,G5020,S,3293232,0,+35.0282414,-090.0009887,"POLYGON ((-90.01257 35.02328, -90.01256 35.023..."
1493,47,073,050501,47073050501,505.01,Census Tract 505.01,G5020,S,83253173,0,+36.5659218,-082.7714427,"POLYGON ((-82.85011 36.54107, -82.84955 36.541..."
1494,47,073,050601,47073050601,506.01,Census Tract 506.01,G5020,S,23852225,177932,+36.5836443,-082.6318663,"POLYGON ((-82.68939 36.58793, -82.68936 36.588..."
1495,47,073,050602,47073050602,506.02,Census Tract 506.02,G5020,S,23381992,668716,+36.5467218,-082.6458382,"POLYGON ((-82.69446 36.54856, -82.69445 36.548..."


In [4]:
Incidents['Incident Reported'] =  pd.to_datetime(Incidents['Incident Reported'], infer_datetime_format=True, errors='coerce')

#Incidents['Incident Reported'].dt.date


In [5]:
Incidents

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Weapon Description,Victim Number,Domestic Related,Victim Type,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location
0,20210237359_11,20210237359,D,DISPATCHED,O,OPEN,Open,04/27/2021 08:00:00 AM,2021-04-28 14:08:00,DIVISION ST,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Hispanic,NON RESIDENT,POINT (-86.78 36.15)
1,20210248988_12,20210248988,D,DISPATCHED,O,OPEN,Open,05/04/2021 03:07:00 PM,2021-05-04 16:26:00,SHELBY AVE,...,NONE,2,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,POINT (-86.75 36.17)
2,20150345704_11,20150345704,T,NaN,O,OPEN,Open,04/13/2015 03:00:00 PM,2015-04-13 17:12:00,180 WALLACE RD,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,NaN
3,20210247765_11,20210247765,O,NaN,O,OPEN,Open,05/03/2021 11:00:00 PM,2021-05-03 23:49:00,CLARKSVILLE PIKE,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,POINT (-86.9 36.32)
4,20210249540_11,20210249540,D,DISPATCHED,O,OPEN,Open,05/04/2021 11:45:00 PM,2021-05-05 00:45:00,UNIVERSITY CT,...,PERSONAL (HANDS),1,False,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.77 36.15)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884525,20160600990_11,20160600990,D,DISPATCHED,D,DEATH OF OFFENDER,Closed,07/04/2016 03:59:00 PM,2016-07-04 17:59:00,203 203,...,NONE,1,False,S,SOCIETY,NaN,NaN,NaN,NaN,POINT (-86.701 36.27)
884526,20150722886_11,20150722886,D,DISPATCHED,A,CLEARED BY ARREST,Closed,08/01/2015 05:11:00 PM,2015-08-01 17:11:00,405 405,...,PERSONAL (HANDS),1,False,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.801 36.223)
884527,20200775916_11,20200775916,D,DISPATCHED,U,UNFOUNDED,Closed,12/18/2020 06:06:00 PM,2020-12-18 21:15:00,914 914,...,NONE,1,False,U,UNKNOWN,U,U,Non-Hispanic,NaN,POINT (-86.747 36.187)
884528,20150013134_11,20150013134,O,NaN,U,UNFOUNDED,Closed,01/03/2015 08:30:00 AM,2015-01-05 11:28:00,2407 2407,...,NONE,1,False,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.675 36.308)


In [6]:
Incidents['Incident Reported'].isna().sum()

12